# Assginment 22

## Imports

In [1]:
import pandas as pd 
import numpy as np
import pysqlite3 as sql
import re

## Creating the connection 

In [2]:
db = sql.connect("Db-IMDB.db")

## Question 1

### Loading the director data

In [76]:
director_query = '''
select p.Name as director, m_d.MID as MID 
    from ((select Name,PID 
        from Person) p 
    join (select MID,PID 
            from M_Director) m_d 
    on ( m_d.PID = p.PID))
'''
df_dir = pd.read_sql_query(director_query,db)
df_dir.drop_duplicates(subset=['MID','director'],keep=False,inplace=True)
df_dir.set_index('MID',inplace=True)

### Loading the movie data

In [77]:
movie_query = '''
 select MID,year from Movie where MID in ( select MID from M_Genre where GID in (
 select GID from Genre where lower(Name) like '%comedy%'))
 '''
movie_df = pd.read_sql_query(movie_query,db)
movie_df.head()
# Taking care of problematic date outside the db
# Removing the special symbols from date
movie_df['year'] = movie_df['year'].apply(lambda x : re.sub('^[A-z]+','',x))
# Converting the string year to integer
movie_df['year'] = movie_df['year'].apply(lambda x : int(x))
# applying the condition for leap year
movie_df = movie_df[(movie_df['year'] % 4 == 0 ) | (movie_df['year'] % 400 == 0)]
movie_df.set_index('MID',inplace=True)
movie_df.head()

year
MID            
tt3726012  2016
tt0366551  2004
tt1954470  2012
tt0327437  2004
tt0809504  2008

### Joining the two dataframes

In [78]:
ans_df = movie_df.join(df_dir,on='MID',how='inner')

In [80]:
ans_df.drop_duplicates(subset=['year','director'],keep=False,inplace=True)
ans_df

year               director
MID                                   
tt3726012  2016           Milap Zaveri
tt3726012  2016           Milap Zaveri
tt0366551  2004           Danny Leiner
tt0366551  2004           Danny Leiner
tt1954470  2012         Anurag Kashyap
...         ...                    ...
tt0438153  2004  Siddharth Anand Kumar
tt1582597  2008        Amma Rajasekhar
tt1087526  2008          Oliver Paulus
tt2356426  2012            Raja Chanda
tt0363011  1996       K.S. Prakash Rao

[366 rows x 2 columns]